In [1]:
import numpy as np
import pandas as pd
from __future__ import division
import matplotlib.pyplot as plt

## Part 1: KNN

### Exercise: K-nearest-neighbors map.

In [8]:
df = pd.read_csv("../week_3/SFPD_Incidents_-_from_1_January_2003.csv")			 # read our dataset
df = df.dropna()																 # drop all NaN values
columns = ["Category", "Date", "Time", "PdDistrict", 'X', 'Y']
df = df[columns]																 # select columns we're interested in
df.head()

,Category,Date,Time,PdDistrict,X,Y
0,NON-CRIMINAL,01/19/2015,14:00,MISSION,-122.421582,37.761701
1,ROBBERY,02/01/2015,15:45,TENDERLOIN,-122.414406,37.784191
2,ASSAULT,02/01/2015,15:45,TENDERLOIN,-122.414406,37.784191
3,SECONDARY CODES,02/01/2015,15:45,TENDERLOIN,-122.414406,37.784191
4,VANDALISM,01/27/2015,19:00,NORTHERN,-122.431119,37.800469


* **Begin by using geoplotlib to plot all incidents of the three crime types (PROSTITUTION, DRUG/NARCOTIC and DRIVING UNDER THE INFLUENCE) on their own map using geoplotlib.kde(). This will give you an idea of how the varioius crimes are distributed across the city.**

In [9]:
focuscrimes = sorted(['PROSTITUTION', 'DRUG/NARCOTIC', 'DRIVING UNDER THE INFLUENCE'])

In [10]:
focuscrimes_df = df[df['Category'].isin(focuscrimes)] # make a dataframe with crimes we're interested in

In [11]:
focuscrimes_df.head()

,Category,Date,Time,PdDistrict,X,Y
11,DRUG/NARCOTIC,02/01/2015,14:00,MISSION,-122.413354,37.769075
12,DRUG/NARCOTIC,02/01/2015,14:00,MISSION,-122.413354,37.769075
31,DRUG/NARCOTIC,02/01/2015,16:56,MISSION,-122.419520,37.764229
41,DRUG/NARCOTIC,02/01/2015,17:02,MISSION,-122.419828,37.766674
61,DRIVING UNDER THE INFLUENCE,02/01/2015,18:39,NORTHERN,-122.420692,37.790577


In [12]:
import geoplotlib
from geoplotlib.utils import BoundingBox 
from scipy import stats

# remove obs that are not within 3 stds
focuscrimes_df = focuscrimes_df[(np.abs(stats.zscore(focuscrimes_df[['X','Y']])) < 5).all(axis=1)] 

latitute_list = focuscrimes_df['Y']
longitude_list = focuscrimes_df['X']

geo_data_for_plotting = {"lat": latitute_list,
                         "lon": longitude_list}

In [13]:
max_lat, min_lat = max(latitute_list), min(latitute_list)
max_lon, min_lon = max(longitude_list), min(longitude_list) 

# Select and set the boundaries of our map
bbox = BoundingBox(north=max_lat, west=min_lon, south=min_lat, east=max_lon)
geoplotlib.set_bbox(bbox)

# Do the actual plotting
geoplotlib.kde(geo_data_for_plotting, bw=6, alpha=160, cut_below=1e-3)
geoplotlib.inline()

('smallest non-zero count', 1.453301797931636e-09)
('max count:', 32.775887320889446)


* **Next, it's time to set up your model based on the actual data. You can use the code supplied in the book or try out scikit-learn's KNeighborsClassifier.**


* **You can make the dataset 'balanced' by grabbing an equal number of examples from each crime category. How do you expect that will change the KNN result? In which situations is the balanced map useful - and when is the map that data in proportion to occurrences useful? Choose which map you will work on in the following.**

In [16]:
from collections import Counter

focuscrimes_list = Counter(focuscrimes_df['Category']).most_common()	# grab counts of all focuscrimes
least_common_crime = focuscrimes_list[-1]								# grab the least common crime as baseline 

In [17]:
least_common_crime

('DRIVING UNDER THE INFLUENCE', 5302)

In [18]:
balanced_focuscrimes_df = pd.DataFrame(columns=columns) # initialize empty balanced dataframe

# append to balanced dataframe 
for crime in focuscrimes:
    
    current_df = df[df['Category'] == crime]						 # grab current crime
    current_sample = current_df.sample(n=least_common_crime[-1])	 # randomly sample according to the least common
    
    balanced_focuscrimes_df = pd.concat([balanced_focuscrimes_df, current_sample])

In [19]:
balanced_focuscrimes_df.head()

,Category,Date,Time,PdDistrict,X,Y
416161,DRIVING UNDER THE INFLUENCE,11/30/2003,00:05,TARAVAL,-122.474954,37.732456
1656872,DRIVING UNDER THE INFLUENCE,09/03/2011,19:54,MISSION,-122.426936,37.758211
31058,DRIVING UNDER THE INFLUENCE,04/24/2014,00:41,MISSION,-122.417333,37.763572
1139633,DRIVING UNDER THE INFLUENCE,01/09/2011,00:01,CENTRAL,-122.401857,37.796626
1755858,DRIVING UNDER THE INFLUENCE,10/14/2012,03:04,TARAVAL,-122.483569,37.723065


In [20]:
balanced_focuscrimes_df.shape

(15906, 6)

In [21]:
# Make the column categorical in pandas, so we can convert it to numerical values
balanced_focuscrimes_df['Category'] = balanced_focuscrimes_df['Category'].astype('category')
balanced_focuscrimes_df['Class'] = balanced_focuscrimes_df['Category'].cat.codes

In [22]:
balanced_focuscrimes_df.head()

,Category,Date,Time,PdDistrict,X,Y,Class
416161,DRIVING UNDER THE INFLUENCE,11/30/2003,00:05,TARAVAL,-122.474954,37.732456,0
1656872,DRIVING UNDER THE INFLUENCE,09/03/2011,19:54,MISSION,-122.426936,37.758211,0
31058,DRIVING UNDER THE INFLUENCE,04/24/2014,00:41,MISSION,-122.417333,37.763572,0
1139633,DRIVING UNDER THE INFLUENCE,01/09/2011,00:01,CENTRAL,-122.401857,37.796626,0
1755858,DRIVING UNDER THE INFLUENCE,10/14/2012,03:04,TARAVAL,-122.483569,37.723065,0


* **Now create an approximately square grid of point that runs over SF. You get to decide the grid-size, but I recommend somewhere between $50 \times 50$ and $100 \times 100$ points. I recommend plotting using geoplotlib.dot().**

In [23]:
focuscrimes

['DRIVING UNDER THE INFLUENCE', 'DRUG/NARCOTIC', 'PROSTITUTION']

In [24]:
balanced_focuscrimes_df.shape

(15906, 7)

In [25]:
# Select and set the boundaries of our map
bbox = BoundingBox(north=max_lat, west=min_lon, south=min_lat, east=max_lon)
geoplotlib.set_bbox(bbox)

colors = ["blue", "red", "green"] # colors used to plot our crimes

# Loop through all our focuscrimes
for idx, crime in enumerate(focuscrimes):
    
    # Selecting location for each type of crime
    latitute_list = balanced_focuscrimes_df[balanced_focuscrimes_df['Category'] == crime]['Y']
    longitude_list = balanced_focuscrimes_df[balanced_focuscrimes_df['Category'] == crime]['X']
    
    geo_data_for_plotting = {"lat": latitute_list,
                             "lon": longitude_list}
    
    # And plotting it
    geoplotlib.dot(geo_data_for_plotting, color=colors[idx], point_size=2)


geoplotlib.inline()

In [26]:
# KNN Model setup

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric

X = balanced_focuscrimes_df[['X', 'Y']]						 # feature values
y = balanced_focuscrimes_df['Class']						 # target

knn = KNeighborsClassifier(n_neighbors=5)					 # initialize model
knn.fit(X, y)												 # fit to our data

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

When looking at the distances between data points and their closest neighbours - an interesting observation is that most of them actually have a distance of zero to their nearest neighbours. Even excluding the distance of a given point to itself, the distances are 0. This indicates that a collection of crimes is marked as happening *exactly* at the same location.

As can be seen below, the total number of distances (in the case if $K = 5$ is $5 \times 15342$). As the KNN is also measuring a point's distance to itself (which is 0), we would therefore expect $4 \times 15342$ non-zero values.

In [27]:
all_distances = knn.kneighbors(X=X, n_neighbors=5, return_distance=True)	 # grab all distances (15342, 5)
all_distances[0].shape

(15906, 5)

However, looking at the distances that are not equal to 0, we can see that their total amount is 20576.

In [28]:
all_distances[0][all_distances[0] != 0].shape

(20860,)

This indicates that many of our observations are being marked as happening at the exact same location. For example, looking at observation 4 in more detail we can see that it's distances are 0 to all it's neighbours:

In [29]:
knn.kneighbors(X=X.iloc[4,:].values.reshape(1,-1), n_neighbors=5, return_distance=True)[0] # grab the distances for obs. 4

array([[ 0.,  0.,  0.,  0.,  0.]])

Looking at those neighbours in detail, we can see that despite happening at differnet times, they are being marked at exactly the same spot down to the 14h decimal (-122.48356927083799, 37.723065055678099). 

Looking at the exact coordinates one can see that this is close to the Font Blvd & Mary Ward Hall bus station. Therefore, it can be assumed that the police is marking all crimes happening in that relative at the coordinates mentioned above. Therefore, the reason many of the distances in our input data for the KNN model is market as 0 is due to the uncertainty when marking the crime location.

In [32]:
balanced_focuscrimes_df[balanced_focuscrimes_df['X'] == X.iloc[4,:][0]]

,Category,Date,Time,PdDistrict,X,Y,Class
1755858,DRIVING UNDER THE INFLUENCE,10/14/2012,03:04,TARAVAL,-122.483569,37.723065,0
852240,DRIVING UNDER THE INFLUENCE,09/05/2011,00:34,TARAVAL,-122.483569,37.723065,0
1867025,DRIVING UNDER THE INFLUENCE,05/23/2016,02:29,TARAVAL,-122.483569,37.723065,0
1764529,DRIVING UNDER THE INFLUENCE,09/05/2011,00:34,TARAVAL,-122.483569,37.723065,0
1876738,DRIVING UNDER THE INFLUENCE,01/23/2016,03:55,TARAVAL,-122.483569,37.723065,0
584082,DRUG/NARCOTIC,02/13/2013,21:22,TARAVAL,-122.483569,37.723065,1
962211,DRUG/NARCOTIC,10/31/2008,16:40,TARAVAL,-122.483569,37.723065,1
1252387,DRUG/NARCOTIC,11/07/2011,22:43,TARAVAL,-122.483569,37.723065,1
1564248,DRUG/NARCOTIC,04/15/2012,22:54,TARAVAL,-122.483569,37.723065,1


Looking at the plot of all crime occurences (figure XXX) one can see that there are some areas of concentrated crime activity and some areas with very sparse crime activities. As KNN is a relatively simple model that assigns to a class based on the minimium distances (without accounting for how large the distances acutally are), plotting the decision boundaries will assign a crime to each point in the grid, no matter how far from an actual crime the point is. For example,  

It should be noted that taking the mean of non zero distances as the threshold a sparse area is somewhat of a arbitrary choice. In order to build a more robust model to predict which category a crime belongs to given that a crime happened $(P(C \mid X, Y)$ where $C \in \{PRO, DUI, DRG\})$ more analysis (such as cross validation) should be done to estimate the threshold.

In [67]:
# Outlier detection for regularization
 
all_distances = knn.kneighbors(X=X, n_neighbors=5, return_distance=True)	 # grab all distances (15342, 5)
mean_dist = np.mean(all_distances[0][all_distances[0] != 0]					 # and the mean of non-zero distances
dist_std = np.std(all_distances[0])											 # and their std
outlier_dist = mean_dist													 # threshold to be considered outlier

In [68]:
outlier_dist

0.00088437935313344493

In [69]:
# Plotting

# Initialize map locationl
bbox = BoundingBox(north=max_lat, west=min_lon, south=min_lat, east=max_lon)
geoplotlib.set_bbox(bbox)

colors = ["blue", "red", "green"] # DUI, DRUGS, PROSTITUTION

lat_distance = max_lat - min_lat # vertical span of grid
lon_distance = max_lon - min_lon # horzontal span of grid 

lat_step = lat_distance/100 # vertical spacing between points
lon_step = lon_distance/100 # horizontal spacing between points


# Initialize dictionaries for different crimes
geo_data_for_plotting_dui = {"lat": [],
                             "lon": []}

geo_data_for_plotting_drg = {"lat": [],
                             "lon": []}

geo_data_for_plotting_prs = {"lat": [],
                             "lon": []}

# for each point in our 100x100 grid
for i in np.arange(min_lat, max_lat, lat_step): # iterate through LAT
    for j in np.arange(min_lon-0.03, max_lon+0.01, lon_step): # iterate through LON
        
        # grab the current grid point and its distances to its KNNs
        current_pred = np.array([[j, i]])
        current_distances = knn.kneighbors(X=current_pred, n_neighbors=5, return_distance=True)[0]        
        
        # check if it's in a sparse area or not
        if np.mean(current_distances) > outlier_dist:
            current_pred = ''
        else:
            current_pred = knn.predict(current_pred)     
        
        if current_pred == 0:							# DUI
            geo_data_for_plotting_dui['lat'].append(i)
            geo_data_for_plotting_dui['lon'].append(j)
        elif current_pred == 1:							# DRUGS
            geo_data_for_plotting_drg['lat'].append(i)
            geo_data_for_plotting_drg['lon'].append(j)
        elif current_pred == 2:							# PROSTITUTION
            geo_data_for_plotting_prs['lat'].append(i)
            geo_data_for_plotting_prs['lon'].append(j)

geoplotlib.dot(geo_data_for_plotting_dui, color="blue", point_size=1.5)
geoplotlib.dot(geo_data_for_plotting_drg, color="red", point_size=1.5)
geoplotlib.dot(geo_data_for_plotting_prs, color="green", point_size=1.5)
            
geoplotlib.inline()

KNN
Linear Regression
Mutlivariate 

## Part 2: Linear Regression (DSFS Chapter 14)

## Part 3: Building an understanding of Multiple Regression and Logistic Regression

## Part 4: Use regression models

### Exercise: Fun with linear regression.

First - we build a dataframe with the to be used in the univariate regression

In [ ]:
df.loc[:, 'Year'] = df.loc[:,'Date'].apply(lambda x: x.split("/")[-1])
df.loc[:, 'Hour'] = df.loc[:,('Time')].apply(lambda x: x.split(":")[0])		 # a column with first elem of time (H)
df = df[df['Hour'].isin(hours)]											 # grab only obs betwee 05-17

In [ ]:
# Training period (dataset) is 2015
df_2015 = df[df['Year'] == '2015']
df_2015.head()

In [ ]:
crimes_hours_2015 = Counter(df_2015['Hour'])
crimes_hours_2015

* **We start by picking an area and a crime type, I'm starting with all of SF and LARCENY/THEFT just to have a lots of data, but you're free to try different crime types and areas. We will use 2015 data for training.**

In [ ]:
df_regr = df.loc[df['Category'] == 'LARCENY/THEFT', :]								 # grab only theft
df_regr.loc[:, 'Year'] = df_regr.loc[:,'Date'].apply(lambda x: x.split("/")[-1])	 # add the year of crime to df_regr

In [ ]:
df_regr = df_regr[df_regr['Year'] == '2015']	 # grab only obs from 2015

In [ ]:
df_regr.head()

In [ ]:
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [ ]:
#start_date = date(2015, 1 , 1)
#end_date = date(2016, 1, 1)

univ_regr_df = pd.DataFrame()

# Build dataframe to do the univariate linear regression

for hour in hours:																	 # loop through all our hours 
      
    current_dict = Counter(df_2015[df_2015['Hour'] == hour]['Date'])				 # grab the hours and counts
    
    for single_date in daterange(start_date, end_date):								 # fill missing values with 0
        formatted_single_date = single_date.strftime("%m/%d/%Y")
        if formatted_single_date not in current_dict:
            current_dict[formatted_single_date] = 0
            
    current_hour_df = pd.DataFrame(current_dict.items(), columns=['Date', 'Count'])	 # temp_df to concat with main
    current_hour_df['Time (x1)'] = int(hour)													 # add the current hour
    
    current_hour_df = current_hour_df.sort_values(by='Date')									 # sort according to date
    
    univ_regr_df = pd.concat([univ_regr_df, current_hour_df])								 # concat with main

univ_regr_df.head()

In [ ]:
univ_regr_df.shape

In [ ]:
# Initializing variables
X_uni_train = univ_regr_df['Time (x1)']
y_uni_train = univ_regr_df['Count']

In [ ]:
y_uni_train.shape

In [ ]:
# Normalizing
X_uni_train_norm = (X_uni_train - X_uni_train.min()) / (X_uni_train.max() - X_uni_train.min())

In [ ]:
# Fitting the data
from sklearn import linear_model

clf_univ = linear_model.LinearRegression()

clf_univ.fit(np.array(X_uni_train_norm).reshape(-1,1),  y_uni_train)

In [ ]:
# Select the first observation - which is at 05 o'clock (0 after normalization)
quick_test = X_uni_train_norm.iloc[0]
quick_test

In [ ]:
# We can see that at for 05 o clock, we predict 7.5405339 crimes
clf_univ.predict(np.array([quick_test]).reshape(1,-1))

In [ ]:
# Define our interecept and slope variables for the univariate regression
alpha, beta_1 = clf_univ.intercept_, clf_univ.coef_

In [ ]:
regression_line_limits = np.array([0,1]) # limits to extend regression lines

fig = plt.figure(figsize=(8, 6)) # initialize the general figure

ax = fig.add_subplot(1,1,1)

ax.scatter(X_uni_train_norm, y_uni_train, c='black')
ax.plot(regression_line_limits, beta_1*regression_line_limits + alpha, '-', c="r") # plot the regression line

ax.set_title("Crimes in SF by time of day (05-17)", fontsize=14)
ax.set_xlabel("Time of day (hours)")
ax.set_ylabel("Crime count")
#ax.set_xticks(X_uni_train)
#ax.set_xlim(4,17)

plt.show()

* **We need variables that have an approximately linear dependance on each other, but the daily crime counts are non-monotonic: they go up and then down again. For that reason, we will only make predictions for the interval 5am-5pm where the crime count for LARCENY/THEFT is more or less linearly increasing.**

In [ ]:
import datetime
hours = [datetime.time(i).strftime('%H') for i in range(5,17)]						 # create a list of hours 

df_regr['Hour'] = df_regr.loc[:,('Time')].apply(lambda x: x.split(":")[0])		 # a column with first elem of time (H)
df_regr = df_regr[df_regr['Hour'].isin(hours)]									 # grab only obs betwee 05-17

In [ ]:
df_regr.head()

**Let's check that there actually is a correlation. The goal is to first create a scatterplot.**

* Put time-of-day (in the range 05:00-17:00) on the x -axis.
* Next you go over the training data and bin and observed number of LARCENY/THEFT crimes per hour. You should have 12 data-points per day of training data.
* Plot the (time-of-day, bin-count) tuples. Does it look like there is a correlation?
* Report the Pearson-correlation.
* Now we can fit a straight line to the data, add the straight line to your scatterplot (similar to fig 14-1 in the book).

In [ ]:
# count crimes according to each hour
crimes_per_hour = Counter(df_regr['Hour'])
crimes_per_hour

In [ ]:
# split it into a lists of times and counts
crimes_per_hour_split = zip(*sorted(crimes_per_hour.items()))
crimes_hours, crimes_counts = crimes_per_hour_split[0], crimes_per_hour_split[1]

In [ ]:
m, b = np.polyfit(range(5,17), crimes_counts, 1) # fit a 1 degree polynomial 
regression_line_limits = np.array([4,18]) # limits to extend regression lines

fig = plt.figure(figsize=(8, 6)) # initialize the general figure

ax = fig.add_subplot(1,1,1)

ax.scatter(crimes_hours, crimes_counts, c='black')
ax.plot(regression_line_limits, m*regression_line_limits + b, '-', c="r") # plot the regression line

ax.set_title("Crimes in SF by time of day (05-17)", fontsize=14)
ax.set_xlabel("Time of day (hours)")
ax.set_ylabel("Crime count")
ax.set_xticks(range(5,17))
ax.set_xlim(4,17)

plt.show()

In [ ]:
# Pearsons correlation
np.corrcoef(range(5, 17), crimes_counts)

### Exercise: Fun with multiple regression.

* **We are going to improve our model by incorporating the total amount of crime in the area we're considering (e.g. the Tenderloin). The idea is that if there is a lot going on a certain day (e.g. a protest or a music festival taking place), then more crime in general will be happening - including more of the crime we're looking at (e.g. DRUG/NARCOTIC).**

Before we can include this in the model, however, we have to think carefully. There are two important points:

    Point 1: Difference with respect to average.

    Point 2: Rescaling. 



#### Point 1: Difference with respect to average.

In [ ]:
df.loc[:, 'Year'] = df.loc[:,'Date'].apply(lambda x: x.split("/")[-1])
df.loc[:, 'Hour'] = df.loc[:,('Time')].apply(lambda x: x.split(":")[0])		 # a column with first elem of time (H)
df = df[df['Hour'].isin(hours)]											 # grab only obs betwee 05-17

In [ ]:
# Training period (dataset) is 2015
df_2015 = df[df['Year'] == '2015']
df_2015.head()

In [ ]:
crimes_hours_2015 = Counter(df_2015['Hour'])
crimes_hours_2015

In [ ]:
# Calculate average number of crimes per hour for 2015
crimes_hours_2015_ave = {}

for crime in crimes_hours_2015:
    crimes_hours_2015_ave[crime] = crimes_hours_2015[crime]/365
    
crimes_hours_2015_ave

In [ ]:
#start_date = date(2015, 1 , 1)
#end_date = date(2016, 1, 1)

# Build dataframe to do the linear regression

def multiv_regr_df_builder(dataframe):
    
    multiv_regr_df = pd.DataFrame()
    
    # Count the occurences of crime that year
    crimes_hours_year = Counter(dataframe['Hour'])
    
    # Calculate average number of crimes per hour for the year
    crimes_hours_year_ave = {}

    for crime in crimes_hours_year:
        crimes_hours_year_ave[crime] = crimes_hours_year[crime]/365

    
    for hour in hours:																	 # loop through all our hours 

        current_average = crimes_hours_year_ave[hour]									 # calculate the average
        current_dict = Counter(dataframe[dataframe['Hour'] == hour]['Date'])			 # grab the hours and counts

        for single_date in daterange(start_date, end_date):								 # fill missing values with 0
            formatted_single_date = single_date.strftime("%m/%d/%Y")
            if formatted_single_date not in current_dict:
                current_dict[formatted_single_date] = 0


        current_c_eff_dict = {}															 # to hold c_eff for given hour


        for date in current_dict:														 # calculate actual c_eff (365)
            current_c_eff_dict[date] = current_dict[date] - current_average

        current_hour_df = pd.DataFrame(current_c_eff_dict.items(), columns=['Date','C_eff (x2)'])	 # temp_df to concat with main
        current_hour_df['Time (x1)'] = int(hour)													 # add the current hour

        current_hour_df = current_hour_df.sort_values(by='Date')									 # sort according to date

        y_counts_df = pd.DataFrame(current_dict.items(), columns=['Date', 'Count (y)'])				 # counts (y) for each day
        current_hour_df = pd.merge(current_hour_df, y_counts_df, on='Date')							 # merge them with current

        multiv_regr_df = pd.concat([multiv_regr_df, current_hour_df])								 # concat with main
        
    return multiv_regr_df
    

multiv_regr_df_2015 = multiv_regr_df_builder(df_2015)

In [ ]:
multiv_regr_df_2015.head()

In [ ]:
# This is our training set, 365*12 observations
multiv_regr_df_2015.shape

#### Point 2: Preparing the data and Rescaling

In [ ]:
# Initializing X_train and y_train
X_train_multi_15 = multiv_regr_df_2015[['C_eff (x2)', 'Time (x1)']]
y_train_multi_15 = multiv_regr_df_2015['Count (y)']

In [ ]:
# Normalizing / rescaling x
X_train_multi_15 = (X_train_multi_15 - X_train_multi_15.min()) / (X_train_multi_15.max() - X_train_multi_15.min())
X_train_multi_15.mean()

In [ ]:
X_train_multi_15.head()

#### Fitting the data with multiple regression

In [ ]:
from sklearn import linear_model

clf_multiv = linear_model.LinearRegression()

clf_multiv.fit(X_train_multi_15,  y_train_multi_15)

In [ ]:
clf_multiv.coef_

In [ ]:
clf_multiv.intercept_

In [ ]:
# First 5 observations - as can be seen the predictions are somewhat close to the actual counts
quick_test = X_train_multi_15.iloc[0:5,:]

clf_multiv.predict(quick_test)

###  Exercise: Evaluate the two models.

First, we will select our test observations, which is the first week in January 2016 and first week in June 2016. In order to make predictions using c_eff, we must first calculate it for our 2016 test set.

In [ ]:
# Select all data from 2016
df_2016 = df[df['Year'] == '2016']

In [ ]:
# Calculate C_eff values for 2016
multiv_regr_df_2016 = multiv_regr_df_builder(df_2016)
multiv_regr_df_2016.head()

In [ ]:
# Normalize
multiv_regr_df_2016['C_eff (x2)'] = (multiv_regr_df_2016['C_eff (x2)'] - multiv_regr_df_2016['C_eff (x2)'].min()) / (multiv_regr_df_2016['C_eff (x2)'].max() - multiv_regr_df_2016['C_eff (x2)'].min())
multiv_regr_df_2016['Time (x1)'] = (multiv_regr_df_2016['Time (x1)'] - multiv_regr_df_2016['Time (x1)'].min()) / (multiv_regr_df_2016['Time (x1)'].max() - multiv_regr_df_2016['Time (x1)'].min())

In [ ]:
multiv_regr_df_2016.head()

In [ ]:
from datetime import timedelta, date

# Create our test sets (week_A and week_B)

# first week in January (A) and first week in June (B)
week_A_list = [single_date.strftime("%m/%d/%Y") for single_date in daterange(date(2016,1,1), date(2016,1,8))] 
week_B_list = [single_date.strftime("%m/%d/%Y") for single_date in daterange(date(2016,6,1), date(2016,6,8))]

# grab their values from the 2016 df
week_A = multiv_regr_df_2016[multiv_regr_df_2016['Date'].isin(week_A_list)]
week_B = multiv_regr_df_2016[multiv_regr_df_2016['Date'].isin(week_B_list)]

In [ ]:
week_A.head()

Now we can finally make predictions about our test sets

In [ ]:
# Grab our inputs for week A
week_A_univ_input = np.array(week_A['Time (x1)']).reshape(-1 , 1)
week_A_multiv_input = week_A[['C_eff (x2)','Time (x1)']]

# Make our predictions for week A
week_A_predictions_univ = clf_univ.predict(week_A_univ_input)
week_A_predictions_multiv = clf_multiv.predict(week_A_multiv_input)

# Grab our inputs for week B
week_B_univ_input = np.array(week_B['Time (x1)']).reshape(-1 , 1)
week_B_multiv_input = week_B[['C_eff (x2)','Time (x1)']]

# Make our predictions for week B
week_B_predictions_univ = clf_univ.predict(week_B_univ_input)
week_B_predictions_multiv = clf_multiv.predict(week_B_multiv_input)

In [ ]:
# Calculate absolute errors for both methods in week A
week_A_errors_univ = np.mean(np.sqrt((week_A['Count (y)'] - week_A_predictions_univ)**2))
week_A_errors_multiv = np.mean(np.sqrt((week_A['Count (y)'] - week_A_predictions_multiv)**2))

# Calculate absolute errors for both methods in week B
week_B_errors_univ = np.mean(np.sqrt((week_B['Count (y)'] - week_B_predictions_univ)**2))
week_B_errors_multiv = np.mean(np.sqrt((week_B['Count (y)'] - week_B_predictions_multiv)**2))

In [ ]:
print("Week A univariate errors: {}".format(week_A_errors_univ))
print("Week A multivariate errors: {}".format(week_A_errors_multiv))

print("Week B univariate errors: {}".format(week_B_errors_univ))
print("Week B multivariate errors: {}".format(week_B_errors_multiv))

In [ ]:
np.mean(np.sqrt((week_A['Count (y)'] - week_A_predictions_multiv)**2))

In [ ]:
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#

In [ ]:
# Grab sum of crimes for all Ts (my bins)
# Calculate no "days" in the interval (12 for time)
# Average number of crimes in a time bin = total amount of crimes/(no observations [count of all crimes])

* **Now, we start by calculating the average number of crimes in each time-bin, $c_{ave}(T)$. In practice, we do this by adding up all crimes happening in the interval throughout the training period and divide by the number of days in the training period).**

In [ ]:
df.head()

In [ ]:
# Check how may crimes we have overall (excluding 2016 - our test set)
total_crimes_hours = Counter(df[(df['Category'] == 'LARCENY/THEFT') & (df['Year'] != '2016')]['Hour'])
total_crimes_hours

total_crimes_hours_split = zip(*sorted(total_crimes_hours.items()))
total_crimes_hours, total_crimes_counts = total_crimes_hours_split[0], total_crimes_hours_split[1]
total_crimes_counts = np.array(total_crimes_counts)

In [ ]:
print(total_crimes_hours)
print(total_crimes_counts)

In [ ]:
# calculate the number of days in our training set
no_training_days = len(np.unique(df[(df['Category'] == 'LARCENY/THEFT') & (df['Year'] != '2016')]['Date']))
no_training_days

In [ ]:
from sklearn.preprocessing import normalize

# Scale by the number of days we have (excluding 2016) and normalize
total_crimes_counts = total_crimes_counts/no_training_days
total_crimes_counts

In [ ]:
total_crimes_counts_norm = normalize(np.array(total_crimes_counts).reshape(1,-1))
total_crimes_counts_norm

Build model with all data excluding 2016